In [1]:
# https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
import numpy as np
# fix random seed for reproducibility
np.random.seed(0)
nb_classes = 4

Using TensorFlow backend.


In [2]:
# data_processed.csv
# load pima indians dataset
df = pd.read_csv('data_processed.csv')
# Get testSet
df_test = df.sample(frac=0.2, random_state=47)
df_test.to_csv('testSet.csv', index=False)
# Subtract to get trainingSet
df_train = df[~df.index.isin(df_test.index)]
df_train.to_csv('trainingSet.csv', index=False)

In [3]:
trainingSet = pd.read_csv('trainingSet.csv')
testSet = pd.read_csv('testSet.csv')

In [4]:
Y_train_raw = np.array(trainingSet['Value'])
trainSet_X = trainingSet.drop('Value', axis=1)
X_train = np.array(trainSet_X)
Y_train  = np.zeros((len(trainSet_X), nb_classes))
print Y_train_raw
for i in range(len(Y_train_raw)):
    Y_train[i][Y_train_raw[i] - 1] = 1.0
print X_train.shape

[1 1 1 ... 4 4 4]
(1141, 921)


In [5]:
Y_test = np.array(testSet['Value'])
testSet_X = testSet.drop('Value', axis=1)
X_test = np.array(testSet_X)
Y_test = Y_test - [1]

In [6]:
# create model
model = Sequential()
model.add(Dense(1024, input_dim=921, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.25))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

#model.add(Dense(2, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# print X_train
# print Y_train
# Fit the model
model.fit(X_train, Y_train, epochs=10, batch_size=10, shuffle=True, class_weight='auto',)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1141/1141 [==============================] - 3s 3ms/step - loss: 0.3607 - acc: 0.8510
Epoch 2/10
1141/1141 [==============================] - 1s 818us/step - loss: 0.2485 - acc: 0.8883
Epoch 3/10
1141/1141 [==============================] - 1s 836us/step - loss: 0.2051 - acc: 0.9069
Epoch 4/10
1141/1141 [==============================] - 1s 824us/step - loss: 0.2064 - acc: 0.9043
Epoch 5/10
1141/1141 [==============================] - 1s 854us/step - loss: 0.1949 - acc: 0.9102
Epoch 6/10
1141/1141 [==============================] - 1s 863us/step - loss: 0.1755 - acc: 0.9213
Epoch 7/10
1141/1141 [==============================] - 1s 829us/step - loss: 0.1550 - acc: 0.9347
Epoch 8/10
1141/1141 [==============================] - 1s 830us/step - loss: 0.1628 - acc: 0.9240
Epoch 9/10
1141/1141 [==============================] - 1s 859us/step - loss: 0.1508 - acc: 0.9347
Epoch 10/10
1141/1141 [==============================] - 1s 863

In [8]:
raw_predictions = model.predict(X_test, batch_size=10, verbose=0)
print raw_predictions
predicted = np.argmax(raw_predictions, axis=1)

[[1.00000000e+00 2.32987674e-08 9.30320708e-13 3.38121449e-15]
 [9.24138963e-01 7.37643465e-02 1.91605417e-03 1.80612959e-04]
 [9.99999046e-01 9.48345928e-07 2.74663792e-10 2.91689307e-12]
 ...
 [9.99989748e-01 1.03019738e-05 1.04298676e-08 2.16292470e-10]
 [8.79972875e-01 1.15617596e-01 3.97034455e-03 4.39187774e-04]
 [9.99967098e-01 3.29406794e-05 5.32159561e-08 1.33487810e-09]]


In [9]:
count = 0
print Y_test
for i in range(len(Y_test)):
    if predicted[i] == Y_test[i]:
        count += 1
print 1.0 * count/len(Y_test)

[0 1 0 0 0 0 1 1 0 1 0 0 0 0 3 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 2
 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 1 1 1 2 1 0 0 1 2 0 0 0 0 0 0 0 0 1 0 3 0 0 0 1 1 0 2 0 0 0 0
 0 1 0 2 0 0 0 2 0 0 0 0 1 2 0 1 0 1 1 2 0 0 0 0 0 0 1 0 2 0 0 3 1 0 0 1 0
 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 2 0 0 1 1 2 1 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 3 0 0 1 0 0 0 0 0 0 1 2 0 0 0 3 2 0 0 0 0 0 1 2 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 1 3 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 3 0 0 1 0 0 1
 0 0 1 1 0 0 0 0 0 0 1 2 0 3 0 0 1 0 1 0 1 0 1 0 2 0]
0.726315789474


In [10]:
# print np.count_nonzero(Y_test)

#testSet['Value'].value_counts()
print testSet['Value'].value_counts()

1    202
2     55
3     20
4      8
Name: Value, dtype: int64


In [11]:
print Y_test

[0 1 0 0 0 0 1 1 0 1 0 0 0 0 3 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 2
 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 1 1 1 2 1 0 0 1 2 0 0 0 0 0 0 0 0 1 0 3 0 0 0 1 1 0 2 0 0 0 0
 0 1 0 2 0 0 0 2 0 0 0 0 1 2 0 1 0 1 1 2 0 0 0 0 0 0 1 0 2 0 0 3 1 0 0 1 0
 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 2 0 0 1 1 2 1 0 0 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 3 0 0 1 0 0 0 0 0 0 1 2 0 0 0 3 2 0 0 0 0 0 1 2 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 1 3 0 0 0 0 0 0 0 2 1 0 0 0 0 0 0 3 0 0 1 0 0 1
 0 0 1 1 0 0 0 0 0 0 1 2 0 3 0 0 1 0 1 0 1 0 1 0 2 0]
